# Langfuse setup

In [1]:
from langfuse import get_client, Langfuse
from langfuse.langchain import CallbackHandler

LF_SECRET_KEY = "sk-lf-3d15d46b-46b1-46ba-9747-6e68d73e3125"
LF_PUBLIC_KEY = "sk-lf-3d15d46b-46b1-46ba-9747-6e68d73e3125"
LF_HOST = "https://cloud.langfuse.com"
Langfuse(
    public_key=LF_PUBLIC_KEY,
    secret_key=LF_SECRET_KEY,
    host=LF_HOST
)
# Initialize Langfuse client
langfuse = get_client()

# Initialize Langfuse CallbackHandler for Langchain (tracing)
langfuse_handler = CallbackHandler()


# Langchain Ollama setup

In [2]:
from langchain_ollama import ChatOllama


llm = ChatOllama(
    model="granite4:350m",
    temperature=0,
)

# save graph util

In [3]:
def save_graph_to_file(runnable_graph, output_file_path):
    file_path = f"{output_file_path}.png"
    png_bytes = runnable_graph.get_graph().draw_mermaid_png()
    with open(file_path, 'wb') as file:
        file.write(png_bytes)

# Memory

In [29]:
from langgraph.checkpoint.memory import InMemorySaver

checkpoint = InMemorySaver()

# EXP ------------->>>

## Prompts

In [4]:
Node1_prompt = """
You're a assistance tasked to answer the user question.
Use the tools if needed to generate the answer the question.
Once you have the answer return the answer to the user and explain why and what tool you used to generate the answer.
"""

## Tools

In [46]:
from langchain.tools import tool

@tool
def addition_tool(arg1, arg2):
    """
    This tool performs addition between two numbers.
    Args:
        arg1: first number
        arg2: second number
    """
    print("inside addition tool")
    return arg1 + arg2

@tool
def subtraction_tool(arg1, arg2):
    """
    This tool performs subtraction between two numbers.
    Args:
        arg1: first number
        arg2: second number
    """
    print("inside subtraction tool")
    return arg1 - arg2


## Agent

In [10]:
from langchain.agents import create_agent

node1_agent = create_agent(
    model = llm,
    system_prompt = Node1_prompt,
    tools = [addition_tool, subtraction_tool],
    checkpointer = checkpoint
)

In [7]:
config = {
    "configurable": {"thread_id": "1"},
    "callbacks":[langfuse_handler]
}

In [24]:
response = node1_agent.invoke(
    {"messages": [{"role": "user", "content": "what is 1 + 29"}]},
    config=config,
    # context=Context(user_id="1")
)

inside addition tool


## As graph

In [31]:
from langgraph.graph import StateGraph, START, END

### State

In [32]:
from langgraph.graph import MessagesState

class CustomState(MessagesState):
    """
    Custom state for the graph.
    """
    llm_calls : int
    resp : int


### Node

In [47]:
from langgraph.types import Command

def node1(state: CustomState):
    node1_agent = create_agent(
        model = llm,
        system_prompt = Node1_prompt,
        tools = [addition_tool, subtraction_tool],
        checkpointer = True
    )
    usr_mess =  state["messages"][-1].content
    print(f"input user message : {usr_mess}")
    response = node1_agent.invoke(
        {"messages": [{"role": "user", "content": usr_mess}]},
        # config=config,
        # context=Context(user_id="1")
    )
    ans = response["messages"][-1].content
    state_to_update = {
        "resp" : ans,
        "llm_calls" : 1
    }
    return Command(
        update = state_to_update,
        goto="__end__"
    )

In [48]:
graph_builder = StateGraph(CustomState)

graph_builder.add_node("node1", node1)

graph_builder.add_edge(START, "node1")
# graph_builder.add_edge("node1", END)

In [65]:
from langgraph.checkpoint.memory import InMemorySaver

checkpointer = InMemorySaver()

In [66]:
graph = graph_builder.compile(checkpointer = checkpointer)

In [70]:
from langchain.messages import HumanMessage, AIMessage
question = "What is my name ? "
user_q = HumanMessage(content=question)

In [71]:
final_ans = graph.invoke(
    {
        "messages": [user_q]
    },
    config = {
        "configurable": {
            "thread_id": "2",
        },
        "callbacks": [langfuse_handler],
        "metadata": {
            "langfuse_session_id": "ses1",
            "langfuse_user_id": "usr1",
        },
    }
)
            

input user message : What is my name ? 


In [72]:
final_ans

{'messages': [HumanMessage(content='Hy my name is luke how are you ? ', additional_kwargs={}, response_metadata={}, id='bfc365c2-00dd-4b8b-9c80-d0a5e169099b'),
  HumanMessage(content='What is my name ? ', additional_kwargs={}, response_metadata={}, id='a71598dc-fe76-4295-9f05-07105837c297')],
 'llm_calls': 1,
 'resp': 'I am a language model trained by IBM Research labs. How can I assist you today?'}